In [0]:
pip install azure-storage-blob azure-storage-file-datalake

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, input_file_name
from azure.storage.blob import BlobServiceClient

# ----------------------------------------------------------------------------------
# 1. SPARK CONFIGURATION & AZURE STORAGE AUTHENTICATION
# ----------------------------------------------------------------------------------

# Define Azure Storage account details
STORAGE_ACCOUNT_NAME = "cdmo"
SAS_TOKEN = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-01-30T12:53:06Z&st=2025-01-30T04:53:06Z&spr=https&sig=AxwIrOD2cUZkmCPUEaby1ZPw%2Bu9DE7nksIUknqfytD8%3D"  # Replace with your SAS token
CONTAINER_NAME = "config"
METADATA_FILE_NAME = "metadata_config_20250130.csv"

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("MetadataDrivenLoads") \
    .getOrCreate()

# Configure Spark to use SAS Token for each relevant container
containers = ["00-landing", "01-bronze", "config"]
for container in containers:
    spark.conf.set(f"fs.azure.sas.{container}.{STORAGE_ACCOUNT_NAME}.blob.core.windows.net", SAS_TOKEN)

# Initialize BlobServiceClient for deleting processed files
blob_service_client = BlobServiceClient(
    account_url=f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net",
    credential=SAS_TOKEN
)

# ----------------------------------------------------------------------------------
# 2. LOAD METADATA FILE FROM BLOB STORAGE (wasbs://)
# ----------------------------------------------------------------------------------

def load_metadata():
    """
    Loads the metadata configuration file from Azure Blob Storage (wasbs://)
    and filters datasets for the 'Bronze' processing layer.

    Returns:
        list: A list of dictionaries containing metadata configurations.
    """
    metadata_path = f"wasbs://{CONTAINER_NAME}@{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{METADATA_FILE_NAME}"

    try:
        print(f"🔄 Attempting to read metadata file from {metadata_path}...")
        metadata_df = spark.read.format("csv").option("header", "true").load(metadata_path)

        # Filter for 'Bronze' layer datasets
        metadata_list = metadata_df.filter(metadata_df.Layer == "bronze").collect()

        print(f"📌 Filtered metadata for 'Bronze' layer: {len(metadata_list)} entries found.")
        return metadata_list

    except Exception as e:
        print(f"❌ Error loading metadata file: {e}")
        raise

# ----------------------------------------------------------------------------------
# 3. DELETE PROCESSED FILES FROM SOURCE (00-LANDING)
# ----------------------------------------------------------------------------------

def delete_processed_files(source_container, source_path):
    """
    Deletes processed CSV files from the source container (00-landing) after 
    successful processing to the Bronze layer.

    Args:
        source_container (str): The container where files are stored (00-landing).
        source_path (str): The directory path where files exist.
    """
    try:
        # Get the source container client
        source_client = blob_service_client.get_container_client(source_container)

        # List all files in the source path
        blobs = source_client.list_blobs(name_starts_with=source_path)

        for blob in blobs:
            # Get blob client and delete file
            blob_client = source_client.get_blob_client(blob.name)
            blob_client.delete_blob()
            print(f"🗑️ Deleted processed file: {blob.name}")

    except Exception as e:
        print(f"❌ Error deleting processed files from {source_path}: {e}")

# ----------------------------------------------------------------------------------
# 4. PROCESS DATA BASED ON METADATA CONFIGURATION
# ----------------------------------------------------------------------------------

def process_data(metadata_list):
    """
    Processes data based on metadata configurations. Reads from Azure Blob Storage, 
    applies transformations, and writes back to the target path.

    Args:
        metadata_list (list): List of metadata configurations.
    """
    for metadata in metadata_list:
        try:
            # Extract metadata details
            source_container = metadata["SourceContainer"]
            source_path = metadata["SourcePath"]
            source_format = metadata["SourceFormat"]
            target_container = metadata["TargetContainer"]
            target_path = metadata["TargetPath"]
            target_format = metadata["TargetFormat"]
            add_timestamp = metadata["AddTimestamp"].lower() == "true"

            # Configure wasbs:// paths
            source_path_wasbs = f"wasbs://{source_container}@{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{source_path}"
            target_path_wasbs = f"wasbs://{target_container}@{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{target_path}"

            print(f"🔄 Processing {source_path_wasbs} ➝ {target_path_wasbs}")

            # Read source data using PySpark
            source_df = (
                spark.read.format(source_format)
                .option("header", "true")
                .load(source_path_wasbs)
                .withColumn("Filename", input_file_name())  # Add filename column
            )

            # Apply transformations if specified
            if add_timestamp:
                source_df = source_df.withColumn("LoadTimestamp", current_timestamp())

            # Write the processed DataFrame to the target location
            source_df.write.format(target_format).mode("append").save(target_path_wasbs)

            print(f"✅ Data successfully saved to {target_path_wasbs}")

            # Delete processed files from 00-landing
            delete_processed_files(source_container, source_path)

        except Exception as e:
            print(f"❌ Error processing data for Source Path '{metadata['SourcePath']}': {e}")

# ----------------------------------------------------------------------------------
# 5. MAIN EXECUTION: LOAD METADATA & PROCESS DATA
# ----------------------------------------------------------------------------------

if __name__ == "__main__":
    metadata_list = load_metadata()
    process_data(metadata_list)


🔄 Attempting to read metadata file from wasbs://config@cdmo.blob.core.windows.net/metadata_config_20250130.csv...
📌 Filtered metadata for 'Bronze' layer: 5 entries found.
🔄 Processing wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/customerfeedback/ ➝ wasbs://01-bronze@cdmo.blob.core.windows.net/customerfeedback/
✅ Data successfully saved to wasbs://01-bronze@cdmo.blob.core.windows.net/customerfeedback/
🗑️ Deleted processed file: data/incoming/customerfeedback/customerfeedback_20250130_045331.csv
🔄 Processing wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/manufacturebatch/ ➝ wasbs://01-bronze@cdmo.blob.core.windows.net/manufacturebatch/
✅ Data successfully saved to wasbs://01-bronze@cdmo.blob.core.windows.net/manufacturebatch/
🗑️ Deleted processed file: data/incoming/manufacturebatch/manufacturebatch_20250130_045331.csv
🔄 Processing wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/productformula/ ➝ wasbs://01-bronze@cdmo.blob.core.windows.net/product

In [0]:
from pyspark.sql import SparkSession

# ----------------------------------------------------------------------------------
# 1. SPARK CONFIGURATION & AZURE STORAGE AUTHENTICATION
# ----------------------------------------------------------------------------------

# Define Azure Storage account details
STORAGE_ACCOUNT_NAME = "cdmo"  # Replace with your actual storage account name
SAS_TOKEN = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-01-30T12:53:06Z&st=2025-01-30T04:53:06Z&spr=https&sig=AxwIrOD2cUZkmCPUEaby1ZPw%2Bu9DE7nksIUknqfytD8%3D"  # Replace with your SAS token
CONTAINER_NAME = "01-bronze"  # Target container

# Initialize Spark session
spark = SparkSession.builder.appName("BronzeLayerValidation").getOrCreate()

# Configure Spark to use the SAS Token for authentication
spark.conf.set(f"fs.azure.sas.{CONTAINER_NAME}.{STORAGE_ACCOUNT_NAME}.blob.core.windows.net", SAS_TOKEN)

# ----------------------------------------------------------------------------------
# 2. VALIDATE TABLE COUNTS IN BRONZE LAYER
# ----------------------------------------------------------------------------------

# List of tables in the bronze layer
tables = [
    "customerfeedback",
    "manufacturebatch",
    "productformula",
    "sales",
    "supplier"
]

def validate_bronze_tables(container_name, tables):
    """
    Validates the record count for each table in the Bronze layer.

    Args:
        container_name (str): The Azure Blob Storage container name.
        tables (list): A list of table names to validate.
    """
    for table in tables:
        # Construct the full wasbs:// path for each table in the bronze layer
        table_path = f"wasbs://{container_name}@{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{table}/"

        print(f"🔍 Validating table: {table} at {table_path}")

        try:
            # Read the table into a DataFrame
            df = spark.read.format("delta").load(table_path)

            # Get the count of records
            record_count = df.count()
            print(f"✅ Table: {table}, Record Count: {record_count}")

        except Exception as e:
            print(f"❌ Error reading table '{table}': {e}")

# ----------------------------------------------------------------------------------
# 3. MAIN EXECUTION
# ----------------------------------------------------------------------------------

if __name__ == "__main__":
    validate_bronze_tables(CONTAINER_NAME, tables)


🔍 Validating table: customerfeedback at wasbs://01-bronze@cdmo.blob.core.windows.net/customerfeedback/
✅ Table: customerfeedback, Record Count: 50
🔍 Validating table: manufacturebatch at wasbs://01-bronze@cdmo.blob.core.windows.net/manufacturebatch/
✅ Table: manufacturebatch, Record Count: 50
🔍 Validating table: productformula at wasbs://01-bronze@cdmo.blob.core.windows.net/productformula/
✅ Table: productformula, Record Count: 20
🔍 Validating table: sales at wasbs://01-bronze@cdmo.blob.core.windows.net/sales/
✅ Table: sales, Record Count: 50
🔍 Validating table: supplier at wasbs://01-bronze@cdmo.blob.core.windows.net/supplier/
✅ Table: supplier, Record Count: 20


In [0]:
# ----------------------------------------------------------------------------------
# 2. VALIDATE TABLE COUNTS IN BRONZE LAYER
# ----------------------------------------------------------------------------------

# List of tables in the Bronze layer
tables = [
    "customerfeedback",
    "manufacturebatch",
    "productformula",
    "sales",
    "supplier"
]

def validate_bronze_tables(container_name, tables):
    """
    Validates the record count for each table in the Bronze layer and displays sample records.

    Args:
        container_name (str): The Azure Blob Storage container name.
        tables (list): A list of table names to validate.
    """
    for table in tables:
        # Construct the full wasbs:// path for each table in the bronze layer
        table_path = f"wasbs://{container_name}@{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{table}/"

        print(f"🔍 Validating table: {table} at {table_path}")

        try:
            # Read the table into a DataFrame
            df = spark.read.format("delta").load(table_path)

            # Get the count of records
            record_count = df.count()
            print(f"✅ Table: {table}, Record Count: {record_count}")

            # Display the first 5 records for validation
            print(f"📊 Displaying records for table: {table}")
            df.display()  # Use show() instead of display() for PySpark

        except Exception as e:
            print(f"❌ Error reading table '{table}': {e}")

# ----------------------------------------------------------------------------------
# 3. MAIN EXECUTION
# ----------------------------------------------------------------------------------

if __name__ == "__main__":
    validate_bronze_tables(CONTAINER_NAME, tables)

🔍 Validating table: customerfeedback at wasbs://01-bronze@cdmo.blob.core.windows.net/customerfeedback/
✅ Table: customerfeedback, Record Count: 50
📊 Displaying records for table: customerfeedback


id,FeedbackID,ProductID,CustomerID,Rating,Comments,FeedbackDate,Filename,LoadTimestamp
0,823f8ebb-2962-4792-9b65-04f4c4efc4ab,1734248b-a2aa-4550-a0e7-b442b6380fae,04917cdb-a5d9-4b5a-a53d-be1ead775fee,1,null,2024-05-14,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/customerfeedback/customerfeedback_20250130_045331.csv,2025-01-30T05:03:46.845+0000
1,b8d706ff-db62-4cf7-ab5c-70c30b5182d5,82a3cc22-be81-4387-a969-d007012af821,2ea6d8ac-0b73-4705-bd0d-e7ae80e239ed,1,null,2024-06-25,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/customerfeedback/customerfeedback_20250130_045331.csv,2025-01-30T05:03:46.845+0000
2,7435f123-6d32-4050-905c-2a1aab21622f,f9c95ae4-4d4e-490d-8c89-1c5f141d8d6f,168ff7d5-a2c7-4c51-a67a-84c2ac22852a,2,null,2024-08-06,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/customerfeedback/customerfeedback_20250130_045331.csv,2025-01-30T05:03:46.845+0000
3,918832ce-1be7-48be-8257-635908af3b4e,7f12b042-a480-47fd-9d10-6a1d857683d7,45c9f848-5c84-4870-ac5f-f697c9471f62,4,null,2025-01-06,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/customerfeedback/customerfeedback_20250130_045331.csv,2025-01-30T05:03:46.845+0000
4,cc3bd229-f85f-4992-a135-99b21548b6ee,d3e64104-9e64-4bca-9e1e-8e07a522d046,5f2dfe65-3372-4aa3-a293-bf274dd730af,2,null,2024-09-08,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/customerfeedback/customerfeedback_20250130_045331.csv,2025-01-30T05:03:46.845+0000
5,5ed9b9b3-7dd7-4a14-a59e-14dda0b583ed,4164f941-ac1e-4c44-80f4-9c6b14288f31,e05cd884-8b2f-44cd-9058-7f5831228596,2,null,2024-03-10,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/customerfeedback/customerfeedback_20250130_045331.csv,2025-01-30T05:03:46.845+0000
6,f65b0900-c1e8-49ac-9df1-46235d81cba1,ea966f4b-23f7-4bb1-b0f4-8054cbccf5e7,89972455-a774-4845-b204-f21eba569be6,3,null,2024-09-11,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/customerfeedback/customerfeedback_20250130_045331.csv,2025-01-30T05:03:46.845+0000
7,badb2b83-42de-4ffa-a710-88c12598cf25,88ddf5c5-b22d-4450-8ae5-f92419a3d2ee,4a79f34a-23ac-4028-84dc-6e04a259e2a1,4,null,2024-11-18,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/customerfeedback/customerfeedback_20250130_045331.csv,2025-01-30T05:03:46.845+0000
8,b3bf7a5d-3409-45fe-b9a1-575cebe6708a,3f8f33b9-2257-4abf-8277-2a39b64fe42d,cfb0df0f-a7c1-4c7c-848f-95c8279fe657,3,null,2024-04-12,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/customerfeedback/customerfeedback_20250130_045331.csv,2025-01-30T05:03:46.845+0000
9,660eaf97-be6b-4fed-b0fa-3faf4ed708d0,87cdd7f9-a33c-478a-bcee-e9836bdf3da7,d159da75-12b7-4807-8e8a-ae0bf8494d88,1,null,2024-02-16,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/customerfeedback/customerfeedback_20250130_045331.csv,2025-01-30T05:03:46.845+0000


🔍 Validating table: manufacturebatch at wasbs://01-bronze@cdmo.blob.core.windows.net/manufacturebatch/
✅ Table: manufacturebatch, Record Count: 50
📊 Displaying records for table: manufacturebatch


id,BatchID,ProductID,BatchDate,Quantity,Status,Filename,LoadTimestamp
0,ca7f56a4-00c7-40d3-8bfe-3685ca6db601,832aaaf3-1c0e-4ee4-9041-a360c35df8a0,2024-08-28,295,Failed,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/manufacturebatch/manufacturebatch_20250130_045331.csv,2025-01-30T05:03:57.245+0000
1,406a93b8-140e-4833-9859-32fdefcd910b,2e075033-d912-4f75-9b8c-29c615eeb057,2024-02-27,205,In Progress,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/manufacturebatch/manufacturebatch_20250130_045331.csv,2025-01-30T05:03:57.245+0000
2,ad3b3640-f7da-4a95-91d0-4ad36d5f8e48,e466811f-7026-4fcd-acae-81b262471226,2024-11-11,917,Completed,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/manufacturebatch/manufacturebatch_20250130_045331.csv,2025-01-30T05:03:57.245+0000
3,ce042c8e-58c9-439d-90d8-1154d224220b,0f93e1cb-0826-41d7-a68b-be6b4c303bc9,2024-11-12,198,Pending,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/manufacturebatch/manufacturebatch_20250130_045331.csv,2025-01-30T05:03:57.245+0000
4,6a7258ba-8398-4877-9d33-c18a17d1a4c4,b3bd3eee-e34b-4215-b3e8-dfab31e9fe05,2024-08-06,303,Failed,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/manufacturebatch/manufacturebatch_20250130_045331.csv,2025-01-30T05:03:57.245+0000
5,c2b441d3-3478-404f-b115-d136d121b89f,dc60d870-5a07-4f58-bb7e-ba290e01c500,2024-06-13,382,In Progress,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/manufacturebatch/manufacturebatch_20250130_045331.csv,2025-01-30T05:03:57.245+0000
6,7698d3ad-ca34-4725-b807-1034eb2f93e8,ae1c0ea0-a83a-42ae-be70-a90a85ddcbd3,2024-10-15,474,Pending,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/manufacturebatch/manufacturebatch_20250130_045331.csv,2025-01-30T05:03:57.245+0000
7,707f4676-8ca5-4700-9728-fc948de16809,cdd1fc56-ab12-45e7-b38b-85898e86dc43,2024-09-03,137,Pending,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/manufacturebatch/manufacturebatch_20250130_045331.csv,2025-01-30T05:03:57.245+0000
8,402d07a8-31e9-4bfc-846a-37857dbbaf6a,f53ac38d-2d99-4279-9286-61134a5c9171,2024-10-22,748,Failed,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/manufacturebatch/manufacturebatch_20250130_045331.csv,2025-01-30T05:03:57.245+0000
9,387eb6a5-7b16-4b72-b9d4-4e99032cd5cd,fd520b43-c1ea-4ee7-8f0f-42c0edb4bf15,2024-04-27,535,In Progress,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/manufacturebatch/manufacturebatch_20250130_045331.csv,2025-01-30T05:03:57.245+0000


🔍 Validating table: productformula at wasbs://01-bronze@cdmo.blob.core.windows.net/productformula/
✅ Table: productformula, Record Count: 20
📊 Displaying records for table: productformula


id,ProductID,ProductName,Category,FormulationType,PrimaryIngredients,LaunchDate,Filename,LoadTimestamp
0,24a310ab-684e-4c77-a9aa-2ae828c1e544,8 Foundation,Lipstick,Liquid,"Hyaluronic Acid, Vitamin E",2024-04-05,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/productformula/productformula_20250130_045331.csv,2025-01-30T05:04:06.856+0000
1,2d711bd7-fd4c-4a74-8e3c-a0dff069c652,6 Lipstick,Lipstick,Liquid,"Shea Butter, Hyaluronic Acid",2023-05-27,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/productformula/productformula_20250130_045331.csv,2025-01-30T05:04:06.856+0000
2,739600ed-69f4-42ae-b91c-f6b2551ae590,1 Lipstick,Eyeshadow,Cream,"Shea Butter, Collagen",2023-12-17,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/productformula/productformula_20250130_045331.csv,2025-01-30T05:04:06.856+0000
3,f2989b72-33c2-431f-8382-250a23ae28ef,4 Eyeshadow,Foundation,Stick,"Collagen, Collagen",2024-08-06,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/productformula/productformula_20250130_045331.csv,2025-01-30T05:04:06.856+0000
4,a80ef484-6b83-4179-aaf5-88207da63468,1 Blush,Lipstick,Liquid,"Collagen, Vitamin E",2023-08-01,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/productformula/productformula_20250130_045331.csv,2025-01-30T05:04:06.856+0000
5,2a3cd631-79aa-4602-a6e8-5a1872d9b17b,0 Lipstick,Lipstick,Powder,"Aloe Vera, Vitamin E",2024-11-26,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/productformula/productformula_20250130_045331.csv,2025-01-30T05:04:06.856+0000
6,a98252a8-2804-43a6-966e-b44b1a7cced8,3 Foundation,Eyeshadow,Liquid,"Collagen, Hyaluronic Acid",2024-08-24,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/productformula/productformula_20250130_045331.csv,2025-01-30T05:04:06.856+0000
7,f51590b3-c45f-43a9-b9e3-58ad9da7281d,4 Eyeshadow,Blush,Liquid,"Aloe Vera, Aloe Vera",2023-11-01,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/productformula/productformula_20250130_045331.csv,2025-01-30T05:04:06.856+0000
8,12c97626-7037-401d-be0f-4ba1fa6a0b05,5 Blush,Eyeshadow,Gel,"Vitamin E, Collagen",2023-09-03,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/productformula/productformula_20250130_045331.csv,2025-01-30T05:04:06.856+0000
9,d911727b-eb71-40ff-9615-54a5a45d3b9e,7 Eyeshadow,Blush,Powder,"Hyaluronic Acid, Vitamin E",2023-07-06,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/productformula/productformula_20250130_045331.csv,2025-01-30T05:04:06.856+0000


🔍 Validating table: sales at wasbs://01-bronze@cdmo.blob.core.windows.net/sales/
✅ Table: sales, Record Count: 50
📊 Displaying records for table: sales


id,OrderID,CustomerID,ProductID,Quantity,TotalAmount,OrderDate,Filename,LoadTimestamp
0,24a3c597-b47c-44f3-9ea3-ce61eb4fbb86,8cc26038-7ac2-4794-9926-dd3a564a66a0,4aef150d-abbe-4777-8006-606043a20910,6,421.44876,2024-06-22,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/sales/sales_20250130_045331.csv,2025-01-30T05:04:15.794+0000
1,3a615482-8a4f-4525-997b-25518915bda1,2249009d-86df-4174-a43a-e7aa349d39d9,b4b7c670-84b1-4190-9007-ccff70bbf977,7,104.24159,2025-01-12,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/sales/sales_20250130_045331.csv,2025-01-30T05:04:15.794+0000
2,c580a924-00f7-4b14-9042-764aaba00fc4,ae5ac3de-29e0-4402-bd8f-bbfb96a92865,56fc973f-660d-40cd-969e-38ae1ef36a96,7,297.30954,2024-07-24,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/sales/sales_20250130_045331.csv,2025-01-30T05:04:15.794+0000
3,625e0563-17c1-4d45-96de-d642777ab8a0,cf958a04-c24a-4444-8505-cefbe81fee3f,2d6b3cfe-e847-43de-b522-46df561c457d,3,360.72583,2024-02-15,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/sales/sales_20250130_045331.csv,2025-01-30T05:04:15.794+0000
4,658bae55-f255-4312-b634-3ba2a9ac7d6c,8b1e9b48-dae0-4ef7-988a-2a80fd0ec5f9,4ed98e1c-de11-4696-ae47-f491fcb5f77d,7,99.40935,2024-11-19,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/sales/sales_20250130_045331.csv,2025-01-30T05:04:15.794+0000
5,80c160fb-9d5a-4fcc-b0a8-e4f437838a37,8f278952-b58f-4b6f-9b9b-1dcc7f316f38,5848cb63-860c-4b39-a92e-5a0476d4f463,1,378.37415,2025-01-15,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/sales/sales_20250130_045331.csv,2025-01-30T05:04:15.794+0000
6,fb2a2e47-d608-4b6d-ac77-b3fdf391f3db,b9b31f50-a41e-4cda-8562-8060636fbe26,4e678800-8830-4b7b-bc19-0b7e93cc26d8,7,154.32324,2024-12-15,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/sales/sales_20250130_045331.csv,2025-01-30T05:04:15.794+0000
7,53fc5511-dbe2-4d40-9876-fafe07219a3b,da823fdd-da14-4cb3-9fce-87838318640d,d5a47bee-2b43-47a3-90d1-e528b59944d2,1,179.61403,2024-09-29,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/sales/sales_20250130_045331.csv,2025-01-30T05:04:15.794+0000
8,b696b657-3d74-47ce-b948-49cde0d403e5,9dd8f7f3-c657-4154-844f-7458851d5eb9,75113be0-d45e-4634-8372-e38d5721da64,8,342.97693,2024-09-29,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/sales/sales_20250130_045331.csv,2025-01-30T05:04:15.794+0000
9,e3ed175e-a36a-4a24-9d7f-a42ba0135315,78687693-e7d6-48af-ac67-89b0601c8c70,a4217b25-289f-40c4-9183-17375a864e2b,8,282.14777,2024-12-31,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/sales/sales_20250130_045331.csv,2025-01-30T05:04:15.794+0000


🔍 Validating table: supplier at wasbs://01-bronze@cdmo.blob.core.windows.net/supplier/
✅ Table: supplier, Record Count: 20
📊 Displaying records for table: supplier


id,SupplierID,SupplierName,Material,Cost,DeliveryDate,Filename,LoadTimestamp
0,2457fee9-132d-47d4-88fd-e04406413c0e,null,Aloe Vera,53.149853,2024-11-14,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/supplier/supplier_20250130_045331.csv,2025-01-30T05:04:24.882+0000
1,3a5b64fd-fbca-44b7-820a-c50f602a0f6d,null,Collagen,92.56125,2025-01-19,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/supplier/supplier_20250130_045331.csv,2025-01-30T05:04:24.882+0000
2,f6598e46-d304-409a-883c-1b5cb653dd10,null,Aloe Vera,35.079376,2024-09-12,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/supplier/supplier_20250130_045331.csv,2025-01-30T05:04:24.882+0000
3,4fa6f523-fa3b-40bf-a34b-092598e5a2b9,null,Collagen,92.13411,2024-03-21,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/supplier/supplier_20250130_045331.csv,2025-01-30T05:04:24.882+0000
4,7d2d14ac-7a44-44b4-91ae-a3fc4be2978b,null,Hyaluronic Acid,10.346646,2024-09-08,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/supplier/supplier_20250130_045331.csv,2025-01-30T05:04:24.882+0000
5,1709d957-0b86-49f7-a5ee-38af66e2554d,null,Aloe Vera,33.812927,2024-11-01,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/supplier/supplier_20250130_045331.csv,2025-01-30T05:04:24.882+0000
6,014ae50c-caec-4323-8cb6-884b4576f1fd,null,Aloe Vera,51.105965,2024-10-13,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/supplier/supplier_20250130_045331.csv,2025-01-30T05:04:24.882+0000
7,26b46eb3-89a2-451a-a13f-21428ee7dc10,null,Collagen,28.509096,2024-03-16,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/supplier/supplier_20250130_045331.csv,2025-01-30T05:04:24.882+0000
8,5f7c0959-574d-40cc-8a8d-2a31bff8dc45,null,Vitamin E,90.7694,2024-02-25,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/supplier/supplier_20250130_045331.csv,2025-01-30T05:04:24.882+0000
9,ed256f8a-c13f-4394-ad49-3ca6d6184bca,null,Hyaluronic Acid,93.5862,2024-03-14,wasbs://00-landing@cdmo.blob.core.windows.net/data/incoming/supplier/supplier_20250130_045331.csv,2025-01-30T05:04:24.882+0000
